In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import Ridge
from sklearn.decomposition import FastICA
from sklearn.linear_model import ElasticNet

In [146]:
df = pd.read_csv('/content/ElectricityBill.csv')


numerical_cols = ['Construction_Year', 'Number_of_Floors', 'Energy_Consumption_Per_SqM',
                   'Water_Usage_Per_Building', 'Waste_Recycled_Percentage', 'Occupancy_Rate',
                   'Indoor_Air_Quality', 'Smart_Devices_Count', 'Maintenance_Resolution_Time',
                   'Energy_Per_SqM', 'Number_of_Residents']

categorical_cols = ['Building_Type', 'Green_Certified', 'Building_Status', 'Maintenance_Priority']

# Fill missing values....... fill with mean median mode depending on data type

# For numerical columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())
# For categorical columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Label encoding for categorical columns
label_encoders = {}
for cat_col in categorical_cols:
    le = LabelEncoder()
    df[cat_col] = le.fit_transform(df[cat_col])
    label_encoders[cat_col] = le


# Defining the features and target variable
X = df.drop('Electricity_Bill', axis=1)
Y = df['Electricity_Bill']

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [147]:
# Initialize ElasticNet model
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)

# Train the model
elastic_net.fit(X_train, Y_train)

# Predict on training and test data
Y_train_pred = elastic_net.predict(X_train)
Y_test_pred = elastic_net.predict(X_test)

In [152]:
alpha_values = [0.01, 0.1, 1, 10, 100]

# Initialize an empty list to store results
results_list = []

for alpha in alpha_values:
    # Initialize ElasticNet model with current alpha
    elastic_net = ElasticNet(alpha=alpha, l1_ratio=0.5, random_state=42)

    # Train the model
    elastic_net.fit(X_train, Y_train)

    # Predict on test data
    Y_test_pred = elastic_net.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(Y_test, Y_test_pred)
    rmse = mean_squared_error(Y_test, Y_test_pred, squared=False)
    mae = mean_absolute_error(Y_test, Y_test_pred)
    r2 = r2_score(Y_test, Y_test_pred)
    adj_r2 = 1 - (1 - r2) * (len(Y_test) - 1) / (len(Y_test) - X_test.shape[1] - 1)

    print(f"Alpha: {alpha}")
    print(f"  MSE: {mse}")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R2 Score: {r2}")
    print(f"  Adjusted R2 Score: {adj_r2}")
    print("-" * 40)


Alpha: 0.01
  MSE: 24276857.894035783
  RMSE: 4927.155152218751
  MAE: 3842.2249727301682
  R2 Score: 8.505120317725545e-05
  Adjusted R2 Score: -0.06401206089918321
----------------------------------------
Alpha: 0.1
  MSE: 24267611.164532654
  RMSE: 4926.216719200715
  MAE: 3841.121423809543
  R2 Score: 0.000465905393534749
  Adjusted R2 Score: -0.06360679297867455
----------------------------------------
Alpha: 1
  MSE: 24236425.434417684
  RMSE: 4923.050419650167
  MAE: 3835.0470411464416
  R2 Score: 0.0017503828933501664
  Adjusted R2 Score: -0.062239977177589
----------------------------------------
Alpha: 10
  MSE: 24307006.5318607
  RMSE: 4930.213639575946
  MAE: 3837.5891179007876
  R2 Score: -0.0011567105511067766
  Adjusted R2 Score: -0.06533342276592125
----------------------------------------
Alpha: 100
  MSE: 24358231.74488439
  RMSE: 4935.405935167278
  MAE: 3841.697130377902
  R2 Score: -0.0032665740466735205
  Adjusted R2 Score: -0.0675785339214603
--------------------